### **装载云盘**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### **安装tensorflow 2.0**

In [0]:
!pip install tensorflow-gpu==2.0.0-beta1

In [0]:
# tensorlayer 兼容问题
!pip install imgaug==0.2.6

In [0]:
!pip install tensorlayer

### **cd命令**

In [0]:
import os
os.chdir("drive/My Drive/RL_EA/Actor_Critic")

### **查看当前路径**

In [0]:
!pwd

/content/drive/My Drive/RL_EA/Actor_Critic


# Trust Region Policy Optimization


$$
on-Policy\\
discrete\ \&\ continous
$$ 


常规PG的新旧策略在参数空间上接近，但可能会导致性能相差甚远。TRPO主要思想就是通过**KL散度的限制**来让更新操作保持**回报函数单调不减**。

* **原文中定义的回报函数，Q，V，A：**


$$
\eta(\pi)=\mathbb{E}{s_0,a_0,\ldots}\left[ \sum_{t=0}^{\infty}\gamma^tr(s_t) \right] 
$$

$$
Q_\pi (s_t,a_t)=\mathbb{E}{s_{t+1},a_{t+1},\ldots}\left[ \sum_{l=0}^{\infty}\gamma^l r(s_{t+l}) \right]
$$

$$
V_{\pi}(s_t)=\mathbb{E}{a_t,s_{t+1},\ldots} \left[\sum_{l=0}^{\infty}\gamma^l r(s_{t+l}) \right] 
$$

$$
\begin{equation} \begin{split} A_\pi (s,a)&=Q_\pi(s,a)-V_\pi(s) \\&= r(s)+\gamma V_\pi(s')-V_\pi(s) \end{split} \end{equation}
$$
</br>
* 写成增量形式，要保持策略效果单调不减只要**保证后面的哪一项非负即可**(有证明)。

$$
\eta(\widetilde{\pi})=\eta(\pi)+\mathbb{E}{s_0,a_0,\ldots \sim \widetilde{\pi}}\left[ \sum_{t=0}^{\infty}\gamma^t A_\pi(s_t,a_t) \right] 
$$
</br>
* 各种近似操作：

$$
\mbox{设}\  \rho_\pi (s)=P(s_0=s \mid \pi)+\gamma P(s_1=s \mid \pi)+\gamma^2 P(s_2=s \mid \pi)+\dots=\sum_{t=0}^\infty \gamma^t  P(s_t=s \mid \pi) 
$$

$$
\eta(\widetilde{\pi}) = \eta(\pi)+ \sum_s \sum_{t=0}^\infty \gamma^t  P(s_t=s \mid \widetilde{\pi})  \sum_a \widetilde{\pi}(a_t=a \mid s_t) A_\pi(s_t,a_t) 
$$

$$
\eta(\widetilde{\pi}) = \eta(\pi)+  \sum_s  \rho_\widetilde{\pi}(s) \sum_a \widetilde{\pi}(a \mid s) A_\pi(s,a)
$$

* 由于$\widetilde{\pi}$是要学的新策略，比较难计算，因此用旧策略$\pi$来近似，得到：

$$
L_\pi(\widetilde{\pi})=\eta(\pi)+  \sum_s  \rho_{\pi}(s) \sum_a \widetilde{\pi}(a \mid s) A_\pi(s,a)
$$

* 由于$L_{\pi}()$和$\eta()$在$\pi$处一阶导相同，因此在新旧策略相近的情况下其效果相同。经过作者的证明得到关键的不等式：

$$
\eta(\pi_{new})\ge L_{\pi_{old}}\ (\pi_{new})-C\cdot D_{KL}^{max}(\pi_{old},\pi_{new}) 
$$

$$
C=\frac{4\epsilon \gamma}{(1-\gamma)^2} 
$$

$$
\epsilon=\max_{s,a}\left | A_\pi(s,a) \right|
$$

* 换下角标：

$$
\eta(\pi)\ge L_{\pi_{i}}(\pi)-C\cdot D_{KL}^{max}(\pi_{i},\pi) 
$$

* 并用M来表示下界：

$$
M_i(\pi)=L_{\pi_i}(\pi)-C\cdot D_{KL}^{max}(\pi_{i},\pi)
$$

$$
\eta(\pi_{i+1})\ge M_i(\pi_{i+1})
$$

$$
\eta(\pi_i)=M_i(\pi_i)=L_{\pi_i}(\pi_i) 
$$

* 得到：

$$
\eta(\pi_{i+1})-\eta(\pi_i)\ge M_i(\pi_{i+1})-M_i(\pi_i) 
$$

* **那么$\pi_{i+1}$只要满足下面即可保证单调性：**

$$
\pi_{i+1}=arg\max_\pi \left[  L_{\pi_{i}}(\pi)-C\cdot D_{KL}^{max}(\pi_{i}\parallel \pi)\right] 
$$

* 由于$KL$的max计算复杂，用mean来近似；$C$会限制更新步幅，将其改成有约束的优化；同时用重要性采样来消除对动作概率的求和，**最终将问题改为：**

$$
 \max_{\theta}\mathbb{E}_{s\sim \rho_{\theta_{old}}\ ,\ a\sim \pi_{\theta_{old}}}\left[ \frac{\pi_{\theta}(a \mid s)}{\pi_{\theta_{old}}\ (a\mid s)}A_{\theta_{old}}\ (s,a)\right]\
$$
</br>
$$
subject \ to \quad \mathbb{E}_{s\sim \rho_{\theta_{old}}}\left[ D_{KL}(\pi_{\theta_{old}}\ (\cdot \mid s)\parallel \pi_{\theta}(\cdot \mid s))\right] \le \delta 
$$
</br></br>
在实际优化中用的是**共轭梯度**，而不是SGD（这里用到了**拉格朗是对偶**的方法，因为原问题是个有约束的优化问题，从而想到拉格朗日优化方法）。
</br>
**KL散度可用Fisher矩阵来近似**（极值处二阶导相等），用到**Hession Free**（H阵总是以**Hv**的形式出现，通过两次反向传播来计算）方式减小计算量，得到最终的policy更新公式，$\alpha_{i}$是**linear search**的系数（不断地/2，直到满足约束，或者放弃本次更新）,**得到policy参数的更新方式：**

$$
\theta_{k+1}=\theta_{k}+\alpha^{j}\sqrt{\frac{2\delta}{g^{T}H^{-1}g}}H^{-1}g
$$

对critic网络参数进行更新的方式就是用**MSE（targ-eval），这里的targ用到了GAE（权衡reward考虑多少未来步长）来计算**。

In [0]:
import os
import time
import copy

import matplotlib.pyplot as plt
import numpy as np
import scipy.signal

import gym
import tensorflow as tf
import tensorflow_probability as tfp
import tensorlayer as tl
from gym.spaces import Box, Discrete

In [0]:
##### hyper parameters #####
ENV_NAME = 'Pendulum-v0' # CartPole-v0 Pendulum-v0
SEED = 1
GAMMA = 0.99 # reward折扣因子
EPOCHS = 500 # 500
STEPS_PER_EPOCH = 4000 # 4000
HIDDEN_SIZES = [64] * 2

DELTA = 0.01 # 更新TRPO的KL散度的限制
VF_LR = 1e-3 # value function 学习率
TRAIN_V_ITERS = 80 # 每个epoch中，走这些步就进行梯度下降
DAMPING_COEFF = 0.1 # 数值稳定性相关
CG_ITERS = 10
BACKTRACK_ITERS = 10
BACKTRACK_COEFF = 0.8
LAM = 0.97 # GAE-lambda
MAX_EP_LEN = 1000 # 1000
SAVE_FREQ = 10
EPS = 1e-8 # epsilon

In [0]:
# Utils

def combined_shape(length, shape=None):
  """
  合并length和shape
  """
  if shape is None:
    return length,
  return (length, shape) if np.isscalar(shape) else (length, *shape)

def keys_as_sorted_list(dict):
  return sorted(list(dict.keys()))

def values_as_sorted_list(dict):
  return [dict[k] for k in keys_as_sorted_list(dict)]

def input_layer(dim=None):
  return tl.layers.Input(dtype=tf.float32, shape=combined_shape(None, dim))

def input_layers(*args):
  return [input_layer(dim) for dim in args]

def input_layer_from_space(space):
  '''
  判断输入空间(状态)是连续还是离散
  '''
  if isinstance(space, Box):
    return input_layer(space.shape)
  elif isinstance(space, Discrete):
    return tl.layers.Input(dtype=tf.int32, shape=(None, ))
  raise NotImplementedError

def input_layers_from_spaces(*args):
  return [input_layer_from_space(space) for space in args]

def mlp(x, hidden_sizes=(32, ), activation=tf.tanh, output_activation=None):
  '''
  根据hidden_size:[32,32,4]建网络，for n-1层，return最后一层
  '''
  for h in hidden_sizes[:-1]:
    x = tl.layers.Dense(n_units=h, act=activation)(x)
  return tl.layers.Dense(n_units=hidden_sizes[-1], act=output_activation)(x)

def get_vars(model: tl.models.Model):
  return model.trainable_weights

def count_vars(model: tl.models.Model):
  v = get_vars(model)
  return sum([np.prod(var.shape.as_list()) for var in v])

def gaussian_likelihood(x, mu, log_std):
  '''
  计算正态分布 对数似然函数
  '''
  pre_sum = -0.5 * (((x - mu) / (tf.exp(log_std) + EPS))**2 + 2 * log_std + np.log(2 * np.pi))
  return tf.reduce_sum(pre_sum, axis=1)

def diagonal_gaussian_kl(mu0, log_std0, mu1, log_std1):
  '''
  计算高斯(正态)分布之间的KL散度
  '''
  var0, var1 = tf.exp(2 * log_std0), tf.exp(2 * log_std1)
  pre_sum = 0.5 * (((mu1 - mu0)**2 + var0) / (var1 + EPS) - 1) +log_std1 -log_std0
  all_kls = tf.reduce_sum(pre_sum, axis=1)
  return tf.reduce_mean(all_kls)

def categorical_kl(logp0, logp1):
  '''
  all_kls：KL散度，mean返回论文中用的平均KL散度
  '''
  all_kls = tf.reduce_sum(tf.exp(logp1) * (logp1 - logp0), axis=1)
  return tf.reduce_mean(all_kls)

def flat_concat(xs):
  return tf.concat([tf.reshape(x, (-1, )) for x in xs], axis=0)

def assign_params_from_flat(x, params):
  flat_size = lambda p: int(np.prod(p.shape.as_list()))
  splits = tf.split(x, [flat_size(p) for p in params])
  new_params = [tf.reshape(p_new, p.shape) for p, p_new in zip(params, splits)]
  return tf.group([p.assign(p_new) for p, p_new in zip(params, new_params)])

def discount_cumsum(x, discount):
  return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]

In [0]:
# Policies
class MlpCategoricalPolicy:
  """
  离散动作
  """
  def __init__(self, x, a, hidden_size, activation, output_activation):
    self.act_dim = a.n
    x = input_layer_from_space(x)
    logits = mlp(x, list(hidden_size) + [self.act_dim], activation, None)
    self.model = tl.models.Model(x, logits)
    self.model.train()

  def cal_outputs_0(self, states):
    states = states.astype(np.float32)
    logits = self.model(states)
    logp_all = tf.nn.log_softmax(logits)
    # pi = tf.squeeze(tfp.distributions.Multinomial(1, logits), axis=1) 
    # 加了._sample_n(1)
    # pi = tf.squeeze(tfp.distributions.Multinomial(1, logits)._sample_n(1), axis=1)
    pi = tf.squeeze(tfp.distributions.Multinomial(1, logits)._sample_n(1))
    if pi.ndim == 1:
      pi = tf.expand_dims(pi, axis=0)    
    pi = tf.argmax(pi, axis=1)
    logp_pi = tf.reduce_sum(tf.one_hot(pi, depth=self.act_dim) * logp_all, axis=1)
    info = {'logp_all': logp_all}
    return pi, logp_pi, info, logp_all

  def cal_outputs_1(self, states, actions, old_logp_all):
    pi, logp_pi, info, logp_all = self.cal_outputs_0(states)
    logp = tf.reduce_sum(tf.one_hot(actions, depth=self.act_dim) * logp_all, axis=1)
    d_kl = categorical_kl(logp_all, old_logp_all)

    info_phs = {'logp_all': logp_all}
    return pi, logp, logp_pi, info, info_phs, d_kl

class MlpGaussianPolicy:
  '''
  连续动作
  '''
  def __init__(self, x, a, hidden_size, activation, output_activation):
    act_dim = a.shape[0]

    x = input_layer_from_space(x)
    mu = mlp(x, list(hidden_size) + [act_dim], activation, output_activation)
    self.model = tl.models.Model(x, mu)
    self.model.train()

    self._log_std = tf.Variable(-0.5 * np.ones(act_dim, dtype=np.float32))
    self.model.trainable_weights.append(self._log_std)

  def cal_outputs_0(self, states):
    states = states.astype(np.float32)
    mu = self.model(states)
    std = tf.exp(self._log_std)
    pi = mu + tf.random.normal(tf.shape(mu)) * std
    logp_pi = gaussian_likelihood(pi, mu, self._log_std)

    info = {'mu': mu, 'log_std': self._log_std}

    return pi, logp_pi, info, mu, self._log_std

  def cal_outputs_1(self, states, actions, old_log_std_ph, old_mu_ph):
    pi, logp_pi, info, mu, log_std = self.cal_outputs_0(states)
    logp = gaussian_likelihood(actions, mu, log_std)
    d_kl = diagonal_gaussian_kl(mu, log_std, old_mu_ph, old_log_std_ph)

    info_phs = {'mu': old_mu_ph, 'log_std': old_log_std_ph}

    return pi, logp, logp_pi, info, info_phs, d_kl

In [0]:
# Actor-Critics
def mlp_actor_critic(
    x: 'env.observation_space', a: 'env.action_space', hidden_sizes=(64, 64), activation=tf.tanh,
    output_activation=None
):
  '''
  创建actor和critic
  '''
  if isinstance(a, Box):
    actor = MlpGaussianPolicy(x, a, hidden_sizes, activation, output_activation)
  elif isinstance(a, Discrete):
    actor = MlpCategoricalPolicy(x, a, hidden_sizes, activation, output_activation)
  else:
    raise ValueError('action space type error')

  class Critic:
    
    def __init__(self, obs_space, hidden_layer_sizes, activation_funcs):
      inputs = input_layer_from_space(obs_space)
      self.model = tl.models.Model(inputs, mlp(inputs, list(hidden_layer_sizes) + [1], activation_funcs, None))
      self.model.train()

    def critic_cal_func(self, states):
      states = states.astype(np.float32)
      return tf.squeeze(self.model(states), axis=1)

  critic = Critic(x, hidden_sizes, activation)

  return actor, critic

In [0]:
# GAEBuffer
class GAEBuffer:
  '''
  经验池，用Generalized Adavantage Esitmation计算优势值
  '''

  def __init__(self, obs_dim, act_dim, size, info_shapes, gamma=0.99, lam=0.95):
    self.obs_buf = np.zeros(combined_shape(size, obs_dim), dtype=np.float32)
    self.act_buf = np.zeros(combined_shape(size, act_dim), dtype=np.float32)
    self.adv_buf = np.zeros(size, dtype=np.float32)
    self.rew_buf = np.zeros(size, dtype=np.float32)
    self.ret_buf = np.zeros(size, dtype=np.float32)
    self.val_buf = np.zeros(size, dtype=np.float32)
    self.logp_buf = np.zeros(size, dtype=np.float32)
    self.info_bufs = {k: np.zeros([size] + list(v), dtype=np.float32) for k, v in info_shapes.items()}
    self.sorted_info_keys = keys_as_sorted_list(self.info_bufs)
    self.gamma, self.lam = gamma, lam
    self.ptr, self.path_start_idx, self.max_size = 0, 0, size

  def store(self, obs, act, rew, val, logp, info):
    assert self.ptr < self.max_size
    self.obs_buf[self.ptr] = obs
    self.act_buf[self.ptr] = act
    self.rew_buf[self.ptr] = rew
    self.val_buf[self.ptr] = val
    self.logp_buf[self.ptr] = logp
    for i, k in enumerate(self.sorted_info_keys):
      self.info_bufs[k][self.ptr] = info[i]
    self.ptr += 1

  def finish_path(self, last_val=0):
    '''
    在轨迹结束时用到，计算整个轨迹中的GAE，折扣reward等

    如果轨迹结束了，那么last_val=0
    '''
    path_slice = slice(self.path_start_idx, self.ptr)
    rews = np.append(self.rew_buf[path_slice], last_val)
    vals = np.append(self.val_buf[path_slice], last_val)

    # GAE 计算
    deltas = rews[:-1] + self.gamma * vals[1:] - vals[:-1]
    self.adv_buf[path_slice] = discount_cumsum(deltas, self.gamma * self.lam)

    # rewards-to-go targets的value func
    self.ret_buf[path_slice] = discount_cumsum(rews, self.gamma)[:-1]

    self.path_start_idx = self.ptr

  def get(self):
    '''
    在epoch结束时使用，得到所有buffer里的数据
    '''
    assert self.ptr == self.max_size
    self.ptr, self.path_start_idx = 0, 0

    # 正规化advantage
    adv_mean, adv_std = np.mean(self.adv_buf), np.std(self.adv_buf)
    self.adv_buf = (self.adv_buf - adv_mean) / adv_std
    return [self.obs_buf, self.act_buf, self.adv_buf, self.ret_buf, self.logp_buf]\
     + values_as_sorted_list(self.info_bufs)


In [0]:
## TRPO
class TRPO:

  def __init__(self, obs_space, act_space):

    obs_dim = obs_space.shape
    act_dim = obs_space.shape

    self.actor, self.critic = mlp_actor_critic(obs_space, act_space, HIDDEN_SIZES)

    if isinstance(act_space, Box):
      act_dim = env.action_space.shape[0]
      info_shapes = {'mu': [act_dim], 'log_std': [act_dim]}
    elif isinstance(env.action_space, Discrete):
      act_dim = env.action_space.n
      info_shapes = {'logp_all': [act_dim]}
    else:
      raise Exception('info_shape error')

    self.buf = GAEBuffer(obs_dim, act_dim, STEPS_PER_EPOCH, info_shapes, GAMMA, LAM)

    self.critic_optimizer = tf.optimizers.Adam(learning_rate=VF_LR)

  def get_action_ops(self, states):
    pi, logp_pi, info, *_ = self.actor.cal_outputs_0(states)
    v = self.critic.critic_cal_func(states)
    res0 = [pi, v, logp_pi] + values_as_sorted_list(info)
    res = []
    for i in res0:
      res.append(i + 0) # transfer to tensor
    return res

  def pi_loss(self, inputs):
    # pi_loss
    x_ph, a_ph, adv_ph, ret_ph, logp_old_ph, *info_values = inputs

    pi, logp, logp_pi, info, info_phs, d_kl = self.actor.cal_outputs_1(x_ph, a_ph, *info_values)
    ratio = tf.exp(logp - logp_old_ph) # pi(a|s) / pi_old(a|s)
    pi_loss = -tf.reduce_mean(ratio * adv_ph)
    return pi_loss

  def v_loss(self, inputs):
    # v_loss
    x_ph, a_ph, adv_ph, ret_ph, logp_old_ph, *info_values = inputs
    v = self.critic.critic_cal_func(x_ph)
    v_loss = tf.reduce_mean((ret_ph - v)**2)
    return v_loss

  def train_vf(self, inputs):
    # 训练 v (critic)
    with tf.GradientTape() as tape:
      loss = self.v_loss(inputs)
    grad = tape.gradient(loss, self.critic.model.trainable_weights)
    self.critic_optimizer.apply_gradients(zip(grad, self.critic.model.trainable_weights))

  # 用来做共轭梯度
  def gradient(self, inputs):
    pi_params = self.actor.model.trainable_weights
    with tf.GradientTape() as tape:
      loss = self.pi_loss(inputs)
    grad = tape.gradient(loss, pi_params)
    gradient = flat_concat(grad)
    return gradient

  def hvp(self, inputs, v_ph):
    """
    hessian vector produce
    """
    pi_params = self.actor.model.trainable_weights
    x_ph, a_ph, adv_ph, ret_ph, logp_old_ph, *info_values = inputs

    with tf.GradientTape() as tape1:
      with tf.GradientTape() as tape0:
        pi, logp, log_pi, info, info_phs, d_kl = self.actor.cal_outputs_1(x_ph, a_ph, *info_values)
      g = flat_concat(tape0.gradient(d_kl, pi_params))
      l = tf.reduce_sum(g * v_ph)
    hvp = flat_concat(tape1.gradient(l, pi_params))

    if DAMPING_COEFF > 0:
      hvp += DAMPING_COEFF * v_ph
    return hvp

  def get_pi_params(self):
    pi_params = self.actor.model.trainable_weights
    return flat_concat(pi_params)

  def set_pi_params(self, v_ph):
    pi_params = self.actor.model.trainable_weights
    assign_params_from_flat(v_ph, pi_params)

  def save_ckpt(self):
    if not os.path.exists('model'):
      os.makedirs('model')

    tl.files.save_weights_to_hdf5('model/trpo_actor.hdf5', self.actor.model)
    tl.files.save_weights_to_hdf5('model/trpo_critic.hdf5', self.critic.model)

  def load_ckpt(self):
    tl.files.load_hdf5_to_weights_in_order('model/trpo_actor.hdf5', self.actor.model)
    tl.files.load_hdf5_to_weights_in_order('model/trpo_critic.hdf5', self.critic.model)

  def cg(self, Ax, b):
    '''
    介于最速下降法与牛顿法之间的一个方法，
    它仅需利用一阶导数信息，但克服了最速下降法收敛慢的缺点，
    又避免了牛顿法需要存储和计算Hessian矩阵并求逆的缺点
    '''
    x = np.zeros_like(b)
    r = copy.deepcopy(b) # b-Ax(x) 对于x=0 Ax(x)=0
    p = copy.deepcopy(r)
    r_dot_old = np.dot(r, r)
    for _ in range(CG_ITERS):
      z = Ax(p)
      alpha = r_dot_old / (np.dot(p, z) + EPS)
      x += alpha * p
      r -= alpha * z
      r_dot_new = np.dot(r, r)
      p = r + (r_dot_new / r_dot_old) * p
      r_dot_old = r_dot_new
    return x
  
  def update(self):
    # 准备 hessian func ，gradient eval
    inputs = self.buf.get()
    Hx = lambda x: self.hvp(inputs, x)
    g, pi_l_old, v_l_log = self.gradient(inputs), self.pi_loss(inputs), self.v_loss(inputs)

    # 核心
    x = self.cg(Hx, g)
    alpha = np.sqrt(2 * DELTA / (np.dot(x, Hx(x)) + EPS))
    old_params = self.get_pi_params()

    def set_and_eval(step):
      aa = alpha * x * step
      par = old_params - aa
      self.set_pi_params(par)
      x_ph, a_ph, adv_ph, ret_ph, logp_old_ph, *info_values = inputs
      pi, logp, logp_pi, info, info_phs, d_kl = self.actor.cal_outputs_1(x_ph, a_ph, *info_values)
      loss = self.pi_loss(inputs)
      return [d_kl, loss]

    for j in range(BACKTRACK_ITERS):
      kl, pi_l_new = set_and_eval(step=BACKTRACK_COEFF**j)
      if kl <= DELTA and pi_l_new <= pi_l_old:
        # 接受新参数
        break

      if j == BACKTRACK_ITERS - 1:
        # 线性搜索失败，保持旧参数
        kl, pi_l_new = set_and_eval(step=0.)

    # value func更新
    for _ in range(TRAIN_V_ITERS):
      self.train_vf(inputs)


### **Main （初始化，训练，测试）**

In [0]:
# 初始化
tf.random.set_seed(SEED)
np.random.seed(SEED)

env = gym.make(ENV_NAME)
env.seed(SEED)

agent = TRPO(env.observation_space, env.action_space)

In [0]:
start_time = time.time()
o, r, d, ep_ret, ep_len = env.reset(), 0, False, 0, 0
reward_list = []
for epoch in range(EPOCHS):
  t0 = time.time()
  rew = 0
  for t in range(STEPS_PER_EPOCH):
    agent_outs = agent.get_action_ops(o.reshape(1, -1))
    a, v_t, logp_t, info_t = np.array(agent_outs[0][0], np.float32),\
                   np.array(agent_outs[1], np.float32),\
                   np.array(agent_outs[2], np.float32),\
                   np.array(agent_outs[3:], np.float32)
    # if isinstance(env.action_space, Discrete):
    #   a = a.astype(np.int32)
    agent.buf.store(o, a, r, v_t, logp_t, info_t)

    o, r, d, _ = env.step(a)
    ep_ret += r
    ep_len += 1

    terminal = d or (ep_len == MAX_EP_LEN)
    if terminal or (t == STEPS_PER_EPOCH - 1):
      if not (terminal):
        print("Warnning: trajectory cut off by epoch at %d steps." % ep_len)
      last_val = r if d else agent.critic.critic_cal_func(o.reshape(1, -1))
      agent.buf.finish_path(last_val)
      rew = ep_ret
      o, r, d, ep_ret, ep_len = env.reset(), 0, False, 0, 0
    
  if (epoch % SAVE_FREQ == 0) or (epoch == EPOCHS - 1):
    agent.save_ckpt()

  agent.update()
  print('epoch[{}/{}] ep_ret: {} time: {}'.format(epoch, EPOCHS, rew, time.time() - t0))

  reward_list.append(rew)
  plt.clf()
  plt.ion()
  plt.plot(reward_list)
  plt.title('TRPO ' + str(DELTA))
  plt.ylim(-2000, 0)
  plt.show()
  plt.pause(0.1)
agent.save_ckpt()
plt.ioff()
plt.show()

In [0]:
# test
agent.load_ckpt()
while True:
  o = env.reset()
  for i in range(STEPS_PER_EPOCH):
    # env.render()
    agent_outs = agent.get_action_ops(o.reshape(1, -1))
    a, v_t, logp_t, info_t = agent_outs[0][0], agent_outs[1], agent_outs[2], agent_outs[3:]
    o, r, d, _ = env.step(a)
    if d:
      break

In [0]:
tfp.__version__

'0.7.0'